In [ ]:
##### Extract rank, artist name, song name from Billboards.com #####
import urllib.request as ur
from bs4 import BeautifulSoup
import pandas as pd
import re


url="http://www.billboard.com/charts/greatest-hot-100-singles"
data=BeautifulSoup(ur.urlopen(url), "lxml")
x1=data.find('div', class_='chart-data chart-data--first-week')

ranks=list()
names=list()
artists=list()
for tag in x1.find_all(True):
    if tag.name=='span':
        ranks.append(tag.text)
    if tag.name=='h2':
        names.append(tag.text)
    if tag.name=='h3':        #Some artist names are dead links, hence use 'h3' tag instead of 'a' tag
        artists.append(tag.text)

#Remove \n and spaces in between the artist name        
artists_r=list()
pattern='[\n,\s]+(.+)[\n,\s]+'
regex=re.compile(pattern)
for artist in artists:
    l=regex.findall(artist)
    if l:
        artists_r.append(l[0])   #[0] to get rid of "[]" around each artist name



df=pd.DataFrame({'ranks': ranks, 'songs': names, 'artists' : artists_r})
collist=["ranks","songs","artists"] #To preserve order of columns
df.to_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits.csv", columns=collist, index=False)




##### Cleaning of dataset extracted above to get it in format usable by Echonest API ######
import pandas as pd
import re
df=pd.read_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits.csv", index_col=False)
rows=(df.shape)[0]

pattern1="(.+)\(.*"
regex1=re.compile(pattern1)
pattern2="(.+)with.*"
regex2=re.compile(pattern2)
pattern3="(.+)featuring.*"
regex3=re.compile(pattern3)
pattern4="(.+)&.*"
regex4=re.compile(pattern4)

for i in range(rows):
    l1=regex1.findall(df.ix[i,1].lower())
    l2=regex2.findall(df.ix[i,2].lower())
    l3=regex3.findall(df.ix[i,2].lower())
    l4=regex4.findall(df.ix[i,2].lower())
    if l1:
        df.ix[i,1]=l1[0]
    if l2:
        df.ix[i,2]=l2[0]
    if l3:
        df.ix[i,2]=l3[0]
    if l4:
        df.ix[i,2]=l4[0]

collist=["ranks","songs","artists"] #To preserve order of columns
df.to_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits_cleaned.csv", columns=collist, index=False)



##### Getting song id of each song using Echonest API #####
import urllib.request as ur
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import time

table=pd.read_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits_cleaned.csv")
rows=(table.shape)[0]
table['ids']=0
for i in range(rows): #Get links in correct format for processing (replace spaces by %20)
    table.ix[i,1]=table.ix[i,1].replace(' ',"%20")
    table.ix[i,2]=table.ix[i,2].replace(' ',"%20")


for i in range(rows): 
    print(i)
    url="http://developer.echonest.com/api/v4/song/search?api_key=VGJ6FVZUID01CPIRP&artist=%s&title=%s" %(table.ix[i,2], table.ix[i,1])
    for j in range(2): 
        try: 
            response=ur.urlopen(url)
            break   # No need to try again since data is read (no problems)
        except Exception as e: 
            print("Couldn't do it: %s" %e)
            if(e.msg=="Too Many Requests"): #Case of HTTP Error 429: Too Many Requests
                time.sleep(61) # Sleep then try again one more time (since the j for loop runs twice unless break)
            else: #Case of HTTP Error 400: Bad Request (e.g. song name Me & U; not for cases like artist name has word "featuring")
                j=5 # Set j to a value that won't be taken during this loop (i.e. anything other than 0 and 1)
                break  # No need to try again, since the problem won't be solved unless done manually 
    if(j==5): # Check for case of HTTP Error 400: Bad Request (the else part above)
        continue # Go to next iteration of the i for loop
       
    data=response.read()
    data=data.decode('utf-8') #Converted bytes type to string
    data_j=json.loads(data) #Converted string type to json type
    songs=data_j['response']['songs']
    if songs: #For the case when name not in required format
        table.ix[i,3]=songs[0]['id']

for i in range(rows): #Get back the original names for storing (remove %20) 
    table.ix[i,1]=table.ix[i,1].replace("%20", ' ')
    table.ix[i,2]=table.ix[i,2].replace("%20", ' ')


print(table) 
collist=["ranks","songs","artists", "ids"] #To preserve order of columns
table.to_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits_ids.csv", columns=collist, index=False)




##### use song id to get acoustic features #####
import urllib.request as ur
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import time

table=pd.read_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits_ids.csv")
rows=(table.shape)[0]
table['acousticness']=0
table['danceability']=0
table['duration']=0
table['energy']=0
table['key']=0
table['liveness']=0
table['loudness']=0
table['mode']=0
table['speechiness']=0
table['tempo']=0
table['time_signature']=0
table['valence']=0

for i in range(rows):
    url="http://developer.echonest.com/api/v4/song/profile?api_key=VGJ6FVZUID01CPIRP&id=%s&bucket=audio_summary" %(table.ix[i,3])
    print(i)
    
    for j in range(2):
        try: 
            response=ur.urlopen(url)
            break   
        except Exception as e: 
            print("Couldn't do it: %s" %e)
            if(e.msg=="Too Many Requests"): 
                time.sleep(61) 
            else: 
                j=5 
                break   
    if(j==5): 
        continue

    data=response.read()
    data=data.decode('utf-8') #Converted bytes type to string
    data_j=json.loads(data) #Converted string type to json type
    song=(data_j['response']['songs'])[0]
    if song:
        table.ix[i,4]=song['audio_summary']['acousticness']
        table.ix[i,5]=song['audio_summary']['danceability']
        table.ix[i,6]=song['audio_summary']['duration']
        table.ix[i,7]=song['audio_summary']['energy']
        table.ix[i,8]=song['audio_summary']['key']
        table.ix[i,9]=song['audio_summary']['liveness']
        table.ix[i,10]=song['audio_summary']['loudness']
        table.ix[i,11]=song['audio_summary']['mode']
        table.ix[i,12]=song['audio_summary']['speechiness']
        table.ix[i,13]=song['audio_summary']['tempo']
        table.ix[i,14]=song['audio_summary']['time_signature']
        table.ix[i,15]=song['audio_summary']['valence']

table.to_csv("/Users/Pragya/Documents/PythonProject/music/newdataset/alltimehits_acoustics.csv", index=False)